In [2]:
from el_toolkit.document import Document
from el_toolkit.data_processors import segment_document
from el_toolkit.data_displayer import Displayer
from el_toolkit.lexical_knowledge_base import WordNet_Lexical_Knowledge_Base,Knowledge_Data
from transformers import BertTokenizer
from IPython.display import display, HTML

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)

lkb = WordNet_Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

test_doc = Document.read_json("./tests/test_data/test_docs/without_overlaps/17_mentions.json")

display(HTML(Displayer.display(test_doc,lkb)))
for doc in segment_document(test_doc,biomed_tokenizer,8):
  display(HTML(Displayer.display(doc,lkb)))
  print("-----------------------------------")


Mentions Omitted: 0


-----------------------------------


-----------------------------------


-----------------------------------


In [1]:
from el_toolkit.lkb.wordnet_lkb import WordNet_Lexical_Knowledge_Base

In [2]:
wn = WordNet_Lexical_Knowledge_Base.read_json("data/knowledge_datasets/trend.json")

TypeError: __init__() got an unexpected keyword argument 'id'

In [ ]:
wn = WordNet_Lexical_Knowledge_Base.read_json("data/knowledge_datasets/trend.json")

In [12]:
from el_toolkit.lkb.wordnet_lkb import WordNet_Lexical_Knowledge_Base
kd.lexical_edges[0]

Lexical_Edge(concept_id='C0007404', term_id='S0007487')

In [5]:
from el_toolkit.data_processors import derive_domain
derive_domain(kd,"C0439861","isa")

[Concept(id='C0439861', info={})]

In [8]:
lkb.get_concept_node("C0439861")._outward_related_concept_nodes

[]

In [16]:
lkb.get_concepts_from_term_id("T1")

[Concept(id='C2', info={'tst_attr': 4}),
 Concept(id='C3', info={'tst_attr': 5}),
 Concept(id='C3', info={'tst_attr': 5}),
 Concept(id='C3', info={'tst_attr': 5}),
 Concept(id='C1', info={'tst_attr': 3}),
 Concept(id='C1', info={'tst_attr': 3}),
 Concept(id='C1', info={'tst_attr': 3}),
 Concept(id='C2', info={'tst_attr': 4}),
 Concept(id='C2', info={'tst_attr': 4}),
 Concept(id='C3', info={'tst_attr': 5}),
 Concept(id='C2', info={'tst_attr': 4}),
 Concept(id='C1', info={'tst_attr': 3})]

In [ ]:
Knowledge_Data.read_json("tests/test_data/test_knowledge_data/small_example.json")

In [ ]:
Knowledge_Data.read_json("tests/test_data/test_knowledge_data/small_example.json")

In [ ]:
lkb.get_concept("C1")

In [ ]:
lkb.id_to_concept_node

In [5]:
from

NameError: name 'Concept' is not defined

In [8]:
from el_toolkit.lexical_knowledge_base import *

In [9]:
c = Concept("c1")
t = Term("t1",string="test")

In [10]:
c_node = Concept_Node(c)
c_node._term_nodes

set()